In [ ]:
import torch
from torch import nn
import re
import random
import tqdm
import time

Тексты Ницше (на английском)

In [ ]:
!wget https://s3.amazonaws.com/text-datasets/nietzsche.txt

--2021-01-02 17:33:52--  https://s3.amazonaws.com/text-datasets/nietzsche.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.142.166
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.142.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 600901 (587K) [text/plain]
Saving to: ‘nietzsche.txt’

nietzsche.txt       100%[===================>] 586.82K  --.-KB/s    in 0.02s   

2021-01-02 17:33:52 (30.1 MB/s) - ‘nietzsche.txt’ saved [600901/600901]



In [ ]:
!ls

nietzsche.txt  sample_data


In [ ]:
!head -30 nietzsche.txt

PREFACE


SUPPOSING that Truth is a woman--what then? Is there not ground
for suspecting that all philosophers, in so far as they have been
dogmatists, have failed to understand women--that the terrible
seriousness and clumsy importunity with which they have usually paid
their addresses to Truth, have been unskilled and unseemly methods for
winning a woman? Certainly she has never allowed herself to be won; and
at present every kind of dogma stands with sad and discouraged mien--IF,
indeed, it stands at all! For there are scoffers who maintain that it
has fallen, that all dogma lies on the ground--nay more, that it is at
its last gasp. But to speak seriously, there are good grounds for hoping
that all dogmatizing in philosophy, whatever solemn, whatever conclusive
and decided airs it has assumed, may have been only a noble puerilism
and tyronism; and probably the time is at hand when it will be once
and again understood WHAT has actually sufficed for the basis of such
imposing and abso

In [ ]:
with open('nietzsche.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('length:', len(text))
text = re.sub('[^a-z ]', ' ', text)
text = re.sub('\s+', ' ', text)

length: 600893


In [ ]:
text[:100]

'preface supposing that truth is a woman what then is there not ground for suspecting that all philos'

In [ ]:
INDEX_TO_CHAR = sorted(list(set(text)))
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}

In [ ]:
CHAR_TO_INDEX

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [ ]:
MAX_LEN = 40
STEP = 3
SENTENCES = []
NEXT_CHARS = []
for i in range(0, len(text) - MAX_LEN, STEP):
  SENTENCES.append(text[i: i + MAX_LEN])
  NEXT_CHARS.append(text[i + MAX_LEN])
print('Num sents:', len(SENTENCES))

Num sents: 193075


In [ ]:
print('Vectorization...')
X = torch.zeros((len(SENTENCES), MAX_LEN), dtype=int)
Y = torch.zeros((len(SENTENCES)), dtype=int)
for i, sentence in enumerate(SENTENCES):
  for t, char in enumerate(sentence):
    X[i, t] = CHAR_TO_INDEX[char]
  Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]

Vectorization...


In [ ]:
X[0:1], Y[0]

(tensor([[16, 18,  5,  6,  1,  3,  5,  0, 19, 21, 16, 16, 15, 19,  9, 14,  7,  0,
          20,  8,  1, 20,  0, 20, 18, 21, 20,  8,  0,  9, 19,  0,  1,  0, 23, 15,
          13,  1, 14,  0]]), tensor(23))

In [ ]:
BATCH_SIZE =512
dataset =torch.utils.data.TensorDataset(X, Y)
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()
        
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = nn.Linear(num_hiddens, num_classes)
        
    def forward(self, X):
        out = self.embedding(X)
        _, state = self.hidden(out)
        predictions = self.output(state[0])
        return predictions

In [ ]:
model = NeuralNetwork(nn.GRU, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX))

In [ ]:
model(X[0:1])

tensor([[-0.0117, -0.0156, -0.1968,  0.0008,  0.0055, -0.1993, -0.2989, -0.2344,
          0.2369, -0.1641,  0.1402,  0.3240,  0.2281, -0.1044,  0.0171,  0.0456,
         -0.1063, -0.0041, -0.2305, -0.3968, -0.0650,  0.0014,  0.0148, -0.3102,
          0.1631, -0.0622, -0.0355]], grad_fn=<AddmmBackward>)

In [ ]:
model = model.cuda()

In [ ]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

def generate_text():
    start_index = random.randint(0, len(text) - MAX_LEN - 1)

    generated = ''
    sentence = text[start_index: start_index + MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN), dtype=int)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char]

        preds = model(x_pred.cuda())[0].cpu()
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])

In [ ]:
generate_text()

t least enjoy a healthier sleep we whose|jujrwutmtbgfincmixzdvvgncdceultrzdcgqyoj


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
for ep in range(100):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in tqdm.tqdm_notebook(data):
        X_b, y_b = X_b.cuda(), y_b.cuda()
        optimizer.zero_grad()
        answers = model.forward(X_b)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys



Epoch 0. Time: 4.334, Train loss: 2.113
color it posits as the inducing causes o|ne the furtalamecon pardess and intofech



Epoch 1. Time: 4.072, Train loss: 1.719
titutes all they have hitherto loved a s|eav arus not menuse belo varidy instill 



Epoch 2. Time: 4.084, Train loss: 1.580
 same time have so much spirituality and| vaunty who and all acts and aitportwedi



Epoch 3. Time: 4.051, Train loss: 1.498
doubtful of the freedom of the will even| their iden that to one of success conse


...


Epoch 95. Time: 4.091, Train loss: 1.026
s of novelty as is indeed the changed ga|in around has hand estimate of the aid s



Epoch 96. Time: 4.133, Train loss: 1.029
isease of the will is diffused unequally| must depture mills and with far too men



Epoch 97. Time: 4.084, Train loss: 1.026
lowing leading phases at first single ac|customers produce many honesty that is t



Epoch 98. Time: 4.156, Train loss: 1.027
onstantly applied the terms herd herd in| the notions and itself it is germans va



Epoch 99. Time: 4.148, Train loss: 1.025
 of a new ruling caste for europe they a|re all nee natural fact our need they ha
